In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install pytorch-lightning pytorchvideo torch torchvision gdown


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 50.9 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=b9bf3982cd1ae6e10f92d99712bf1f362b8be53502347caf0dc46d36316d938c
  Stored in directory: /root/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=b4fcf31f7b386b027351bd701879316b7a8ed16a8984e5f167f949e769b6a1ec
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Cre

In [3]:
pip install pytorch-lightning torchvision gdown


Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import requests
import cv2
import torch
import numpy as np
import gdown
from torchvision.io import read_video
from sklearn.metrics.pairwise import cosine_similarity


def download_video(url, output_path):
    """Download the video from a URL (supports Google Drive links using gdown)."""
    if url.startswith("https://drive.google.com"):
        # Google Drive link
        gdown.download(url, output_path, quiet=False)
    else:
        # Generic URL (download via requests)
        response = requests.get(url, stream=True)
        with open(output_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)


def apply_laplacian_filter(frame):
    """Applies a Laplacian filter to a single video frame."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    laplacian = cv2.Laplacian(gray_frame, cv2.CV_64F)  # Apply Laplacian filter
    laplacian = np.uint8(np.absolute(laplacian))  # Get the absolute value and convert back to uint8
    return laplacian


def process_video(video_path, clip_duration=10, resize=(224, 224)):
    """Load video, apply Laplacian filter on frames, and return processed video frames."""
    video, _, _ = read_video(video_path)  # Read video into a tensor (frames, height, width, channels)
    video_frames = []
    
    # Process frames, apply Laplacian filter, and resize
    for i in range(min(clip_duration, video.size(0))):  # Limit to clip_duration frames
        frame = video[i].numpy()  # Convert tensor to numpy array
        laplacian_frame = apply_laplacian_filter(frame)  # Apply Laplacian filter
        resized_frame = cv2.resize(laplacian_frame, resize)  # Resize the frame
        video_frames.append(resized_frame)
    
    return np.array(video_frames)  # Return the processed frames


def video_to_vector(frames):
    """Flatten frames into vectors."""
    return frames.reshape(frames.shape[0], -1)  # Flatten each frame into a vector


def find_nearest_vector(query_vector, vector_set):
    """Find the nearest vector in vector_set to query_vector using cosine similarity."""
    similarity = cosine_similarity(query_vector, vector_set)
    nearest_idx = np.argmax(similarity)  # Find the index of the most similar vector
    return nearest_idx


def main(video_url):
    # Define local path for the downloaded video
    local_video_path = "downloaded_video.mp4"
    
    # Download the video from the URL
    print(f"Downloading video from {video_url}...")
    download_video(video_url, local_video_path)
    print(f"Video downloaded to {local_video_path}")
    
    # Process the video: Get Laplacian transformed frames
    processed_frames = process_video(local_video_path)
    
    # Convert processed frames to vectors
    video_vectors = video_to_vector(processed_frames)
    
    # You may want to create a set of vectors to compare the query video against (example: database of videos)
    # Here we use the first frame as the query for simplicity
    query_vector = video_vectors[0].reshape(1, -1)  # The query vector (could be any frame)
    
    # Find the nearest vector in the video (for this example, it's just finding the first frame)
    nearest_idx = find_nearest_vector(query_vector, video_vectors)
    
    print(f"Nearest frame index: {nearest_idx}")
    print(f"Query vector (first frame): {query_vector}")
    print(f"Nearest vector (frame at index {nearest_idx}): {video_vectors[nearest_idx]}")

if __name__ == "__main__":
    video_url = 'https://drive.google.com/uc?id=1yOL_KOeHKbGjAubxOWB3RU0Qm0Mu6R0d'  # Replace with your video URL
    main(video_url)


Downloading...
From: https://drive.google.com/uc?id=1yOL_KOeHKbGjAubxOWB3RU0Qm0Mu6R0d
To: /kaggle/working/downloaded_video.mp4
100%|██████████| 2.34M/2.34M [00:00<00:00, 4.37MB/s]
/opt/conda/lib/python3.10/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


Video downloaded to downloaded_video.mp4
Nearest frame index: 0
Query vector (first frame): [[ 2  9 19 ... 92 53  4]]
Nearest vector (frame at index 0): [ 2  9 19 ... 92 53  4]
